In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install tensorflow

In [21]:
# Membaca data
data = pd.read_csv('data_balita.csv')

In [22]:
# Menangani missing values, jika ada
data = data.dropna()  # Menghapus baris dengan missing values


In [23]:
# Mapping 'Jenis Kelamin' menjadi angka: laki-laki = 1, perempuan = 0
data['Jenis Kelamin'] = data['Jenis Kelamin'].map({'laki-laki': 1, 'perempuan': 0})


In [24]:
# Mapping 'Status Gizi' menjadi angka: normal = 0, stunted = 1, severely stunted = 2, tinggi = 3
label_encoder = LabelEncoder()
data['Status Gizi'] = label_encoder.fit_transform(data['Status Gizi'])


In [25]:
# Memisahkan fitur dan label
X = data[['Umur (bulan)', 'Jenis Kelamin', 'Tinggi Badan (cm)']].values  # Fitur
y = data['Status Gizi'].values  # Label

In [26]:
# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Menormalisasi data (skala antara -1 hingga 1 atau 0 hingga 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
# Membuat model neural network (Feedforward Neural Network)
model = models.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer 1
    layers.Dense(32, activation='relu'),  # Hidden layer 2
    layers.Dense(4, activation='softmax')  # Output layer (4 kelas: normal, stunted, severely stunted, tinggi)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [29]:
# Kompilasi model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [30]:
# Melatih model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8264 - loss: 0.4753 - val_accuracy: 0.9643 - val_loss: 0.1165
Epoch 2/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9676 - loss: 0.1036 - val_accuracy: 0.9631 - val_loss: 0.0899
Epoch 3/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9726 - loss: 0.0765 - val_accuracy: 0.9747 - val_loss: 0.0688
Epoch 4/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.9765 - loss: 0.0629 - val_accuracy: 0.9813 - val_loss: 0.0527
Epoch 5/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9782 - loss: 0.0568 - val_accuracy: 0.9818 - val_loss: 0.0495
Epoch 6/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9794 - loss: 0.0537 - val_accuracy: 0.9796 - val_loss: 0.0492
Epoch 7/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9792 - loss: 0.0513 - val_accuracy: 0.9859 - val_loss: 0.0396
Epoch 8/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9803 - loss: 0.0478 

In [31]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9818 - loss: 0.0420
Test Accuracy: 98.31%


In [38]:
model.save('model_status_gizi.h5')

In [39]:
import joblib

# Menyimpan scaler
joblib.dump(scaler, 'scaler_status_gizi.pkl')

# Menyimpan label encoder
joblib.dump(label_encoder, 'label_encoder_status_gizi.pkl')

['label_encoder_status_gizi.pkl']

In [40]:
joblib.dump(scaler, 'scaler_status_gizi.pkl')

['scaler_status_gizi.pkl']

In [41]:
# Contoh data baru untuk prediksi
data_baru = np.array([[24, 1, 85]])  # Misalnya Umur=24 bulan, Laki-Laki, Tinggi Badan=85 cm
data_baru_scaled = scaler.transform(data_baru)  # Normalisasi data menggunakan scaler yang sama

# Prediksi status gizi
predictions = model.predict(data_baru_scaled)
predicted_class = np.argmax(predictions, axis=1)[0]  # Ambil kelas yang diprediksi

# Mengonversi hasil prediksi kembali ke label asli
predicted_label = label_encoder.inverse_transform([predicted_class])[0]

print(f"Prediksi Status Gizi: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediksi Status Gizi: normal
